# 🌾 WatsonFarm AI – Smart Farming Assistant
This notebook demonstrates the backend logic for building a smart farming assistant using IBM Watsonx, Watson Discovery, and RAG architecture.

## 🔧 Install Required Libraries

In [ ]:
# Uncomment and run if required
# !pip install ibm-watson openai flask python-dotenv requests

## 📦 Import Required Libraries

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()

## 🔐 Setup IBM Watson and API Credentials

In [ ]:
# Set your IBM service credentials
WATSON_API_KEY = os.getenv("WATSON_API_KEY")
WATSON_DISCOVERY_URL = os.getenv("WATSON_DISCOVERY_URL")
WATSON_TRANSLATOR_URL = os.getenv("WATSON_TRANSLATOR_URL")
GRANITE_API_URL = os.getenv("GRANITE_API_URL")

## 🌐 Translate Farmer's Query to English

In [ ]:
def translate_text(text, source_lang='hi', target_lang='en'):
    url = f"{WATSON_TRANSLATOR_URL}/v3/translate?version=2018-05-01"
    headers = {"Content-Type": "application/json"}
    data = {
        "text": [text],
        "source": source_lang,
        "target": target_lang
    }
    response = requests.post(url, headers=headers, json=data, auth=('apikey', WATSON_API_KEY))
    return response.json()['translations'][0]['translation']

## 🔍 Retrieve Relevant Agricultural Info using Watson Discovery

In [ ]:
def retrieve_documents(query):
    url = f"{WATSON_DISCOVERY_URL}/v2/query"
    params = {
        "project_id": os.getenv("DISCOVERY_PROJECT_ID"),
        "natural_language_query": query,
        "count": 3
    }
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers, params=params, auth=("apikey", WATSON_API_KEY))
    return [doc['text'] for doc in response.json().get('results', [])]

## 🧠 Generate Answer using IBM Granite

In [ ]:
def generate_answer(context, question):
    prompt = f"""You are an AI assistant for farmers. Based on the following information:
{context}

Answer the question: {question}
"""
    payload = {
        "model_id": "granite-13b-chat",
        "input": prompt
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(GRANITE_API_URL, headers=headers, json=payload, auth=("apikey", WATSON_API_KEY))
    return response.json().get("generated_text", "No answer generated.")

## 🌐 Translate Response Back to User's Language

In [ ]:
def translate_back(text, source_lang='en', target_lang='hi'):
    return translate_text(text, source_lang, target_lang)

## ✅ Complete Pipeline: User Query → Answer

In [ ]:
def ask_ai_agent(user_query, user_lang='hi'):
    translated_query = translate_text(user_query, source_lang=user_lang, target_lang='en')
    docs = retrieve_documents(translated_query)
    context = "\n".join(docs)
    response_en = generate_answer(context, translated_query)
    final_answer = translate_back(response_en, source_lang='en', target_lang=user_lang)
    return final_answer